<a href="https://colab.research.google.com/github/mortezaaghajanzadeh/Machine-learning-in-Finance/blob/main/Lecture%201/introduction_to_tensorflow_lecture_1_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture 1 (Part 1): Introduction to TensorFlow in Python.**
### Based on code from Chapter 1 in ``Machine Learning for Economics and Finance in TensorFlow 2'' (Hull, 2021).

In [2]:
# Import libraries.
import tensorflow as tf
import numpy as np
import pandas as pd

## **Listing 1-2.** Implement OLS.

In [3]:
print(tf.__version__)

# Define the data as constants.
X = tf.constant([[1, 0], [1, 2]], tf.float32)
Y = tf.constant([[2], [4]], tf.float32)

# Matrix multiply X by X’s transpose and invert.
beta_0 = tf.linalg.inv(tf.matmul(tf.transpose(X), X))

# Matrix multiply beta_0 by X’s transpose.
beta_1 = tf.matmul(beta_0, tf.transpose(X))

# Matrix multiply beta_1 by Y.
beta = tf.matmul(beta_1, Y)
beta

2.14.0


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [1.]], dtype=float32)>

## **Listing 1-4.** Print tensors.

In [4]:
# Print the feature matrix.
print(X)

# Print the coefficient vector.
print(beta.numpy())

tf.Tensor(
[[1. 0.]
 [1. 2.]], shape=(2, 2), dtype=float32)
[[2.]
 [1.]]


## **Listing 1-6.** Generate OLS predictions with static graphs.

In [5]:
# Define OLS prediction function as static graph.
@tf.function
def ols_predict(X, beta):
	y_hat = tf.matmul(X, beta)
	return y_hat

# Predict Y using X and beta.
predictions = ols_predict(X, beta)

# Print predictions.
print(predictions)

tf.Tensor(
[[2.]
 [4.]], shape=(2, 1), dtype=float32)


## **Listing 1-7.** Solve an OLS model with tf.keras().

In [6]:
# Define sequential model.
ols = tf.keras.Sequential()

# Add dense layer with linear activation.
ols.add(tf.keras.layers.Dense(1, input_shape = (2,),
    use_bias = False , activation = 'linear'))

# Set optimizer and loss.
ols.compile(optimizer = 'SGD', loss = 'mse')

# Train model.
ols.fit(X, Y, epochs = 500)

# Print parameter estimates.
print(ols.weights[0].numpy())

Epoch 1/500
1/1 [==============================] - 0s 139ms/step - loss: 7.3926
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 6.8012
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 6.2678
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 5.7863
Epoch 5/500
1/1 [==============================] - 0s 2ms/step - loss: 5.3514
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 4.9586
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 4.6035
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 4.2823
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 3.9917
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 3.7284
Epoch 11/500
1/1 [==============================] - 0s 2ms/step - loss: 3.4898
Epoch 12/500
1/1 [==============================] - 0s 1ms/step - loss: 3.2734
Epoch 13/500
1/1 [==============================] - 0s 997u

## **Listing 1-8.** Solve an OLS model with tf.estimator().

In [7]:
# Define feature columns.
features = [
tf.feature_column.numeric_column("constant"),
tf.feature_column.numeric_column("x1")
]

# Define model.
ols = tf.estimator.LinearRegressor(features)

# Define function to feed data to model.
def train_input_fn():
	features = {"constant": [1, 1], "x1": [0, 2]}
	target = [2, 4]
	return features, target

# Train OLS model.
ols.train(train_input_fn, steps = 100)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\SE137D~1.520\\AppData\\Local\\Temp\\tmptez0vgco', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_pro

## **Listing 1-9.** Make predictions with an OLS model with tf.estimator().

In [8]:
# Define feature columns.
def test_input_fn():
    features = {"constant": [1, 1], "x1": [3, 5]}
    return features

# Define prediction generator.
predict_gen = ols.predict(input_fn=test_input_fn)

# Generate predictions.
predictions = [next(predict_gen) for j in range(2)]

# Print predictions.
print(predictions)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SE137D~1.520\AppData\Local\Temp\tmptez0vgco\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[{'predictions': array([5.0000067], dtype=float32)}, {'predictions': array([7.000059], dtype=float32)}]


## **Listing 1-10.** List all available devices, select CPU, and then switch to GPU.

In [10]:
# Print list of devices.
devices = tf.config.list_physical_devices()
print(devices)

# Set device to CPU.
tf.config.experimental.set_visible_devices(
devices[0], 'CPU')

# Change device to GPU.
tf.config.experimental.set_visible_devices(
devices[3], 'GPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


ValueError: Unrecognized device: PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

In [11]:
tf.config.list_physical_devices('GPU')

[]